# MobileNetV2 Model

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Paths
dataset_dir = "/content/drive/MyDrive/Potato Disease Leaf Dataset"
train_dir = os.path.join(dataset_dir, "Training")
val_dir   = os.path.join(dataset_dir, "Validation")
test_dir  = os.path.join(dataset_dir, "Testing")

# Parameters
img_size = (224, 224)  # MobileNetV2 default is 224x224
batch_size = 32

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# For validation and test, just preprocess
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load MobileNetV2 base (exclude top classification layers)
base_model = MobileNetV2(input_shape=(224, 224, 3),
                         include_top=False,
                         weights='imagenet')

# Freeze base_model layers
for layer in base_model.layers:
    layer.trainable = False

# Build a new top (classification head)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation='softmax')(x)  # 3 classes

model = models.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train only the new top layers
epochs = 10
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    verbose=1
)

# Optionally, unfreeze some top layers of the base_model for fine-tuning
# For example:
for layer in base_model.layers[-30:]:  # Unfreeze the last 30 layers
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training
fine_tune_epochs = 10
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=fine_tune_epochs,
    verbose=1
)

# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy after Transfer Learning: {test_acc:.4f}")

# Save model
model.save("/content/drive/MyDrive/Potato Disease Leaf CNN Model/potato_leaf_mobilenetv2.h5")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 3251 images belonging to 3 classes.
Found 416 images belonging to 3 classes.
Found 415 images belonging to 3 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 69s 573ms/step - accuracy: 0.7754 - loss: 0.5609 - val_accuracy: 0.9423 - val_loss: 0.1576
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 70s 515ms/step - accuracy: 0.9151 - loss: 0.2387 - val_accuracy: 0.9567 - val_loss: 0.1288
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 81s 507ms/step - accuracy: 0.9440 - loss: 0.1695 - val_accuracy: 0.9736 - val_loss: 0.0998
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 51s 501ms/step - accuracy: 0.9527 - loss: 0.1337 - val_accuracy: 0.9591 - val_loss: 0.1223
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 51s 497ms/step - accuracy: 0.9593 - loss: 0.1364 - val_accuracy: 0.9760 - val_loss: 0.0961
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 84s

Test Accuracy after Transfer Learning: 0.9663
